In [ ]:
# Execute this cell to make this notebook's dependencies available
!pip install -U https://github.com/conda-incubator/condacolab/archive/cuda-version-12.tar.gz
import condacolab
condacolab.install_mambaforge()
!wget -q https://raw.githubusercontent.com/openforcefield/openff-docs/_cookbook_data_main/build/cookbook/colab/openforcefield/openff-toolkit/toolkit_showcase/5tbm_prepared.pdb
!wget -q https://raw.githubusercontent.com/openforcefield/openff-docs/_cookbook_data_main/build/cookbook/colab/openforcefield/openff-toolkit/toolkit_showcase/minim.mdp
!wget -q https://raw.githubusercontent.com/openforcefield/openff-docs/_cookbook_data_main/build/cookbook/colab/openforcefield/openff-toolkit/toolkit_showcase/README.md
!wget -q https://raw.githubusercontent.com/openforcefield/openff-docs/_cookbook_data_main/build/cookbook/colab/openforcefield/openff-toolkit/toolkit_showcase/PT2385.sdf
!wget -q https://raw.githubusercontent.com/openforcefield/openff-docs/_cookbook_data_main/build/cookbook/colab/openforcefield/openff-toolkit/toolkit_showcase/sim.mdp
!wget -q https://raw.githubusercontent.com/openforcefield/openff-docs/_cookbook_data_main/build/cookbook/colab/openforcefield/openff-toolkit/toolkit_showcase/environment.yaml
!mamba env update -q --name=base --file=environment.yaml
from google.colab import output
output.enable_custom_widget_manager()

# Prepare and run a protein-ligand simulation

Building upon the original Open Force Field Toolkit tutorial*, this presentation introduces advanced techniques for preparing Molecular Dynamics simulation. Our enhancements focus on extending the core material to include:

Temperature and Pressure Equilibration: A critical step for realistic simulations, ensuring the system reaches a stable state that mimics real-world conditions.
Custom Reporter for Intermediate Simulation Results: An added feature to save and analyze simulation data at various stages, providing insights into running simulation.
We thank the creators of the original tutorial for their foundational work. Our additions aim to enrich the learning experience, offering participants a more comprehensive understanding of Molecular Dynamics simulations within a notebook environment.

Enjoy the expanded exploration and the new tools at your disposal!
*Original Notebook: https://github.com/openforcefield/openff-toolkit/blob/main/examples/toolkit_showcase/toolkit_showcase.ipynb

In [ ]:
# Imports from the Python standard library
from io import StringIO
from typing import Iterable

# Imports from the comp chem ecosystem
import mdtraj
import nglview
import numpy as np
import openmm
from openff.units import Quantity, unit
from openmm import unit as openmm_unit
from pdbfixer import PDBFixer

from openmm.app import *
from openmm import *
import matplotlib.pyplot as plt

# Imports from the toolkit
from openff.toolkit import ForceField, Molecule, Topology

_(The OpenEye loading warning is expected -- The toolkit is informing us that OETK is unavailable, but it will safely fall back to using RDKit and AmberTools for the same functionality)_

## Introducing the main cast

Merck [provides data] to benchmark Free Energy Perturbation (FEP) procedures. We'll use structures from this dataset for this showcase:

<https://github.com/MCompChem/fep-benchmark>

This example is pre-packaged with one protein-ligand complex from the above repository, however you should be able to download other complexes and run them as well using this workflow. PT2385, our ligand of choice, is an antagonist of the HIF2α protein, a transcription factor that is stabilized in many kidney cancers.

The ligand and protein structures are already prepared for simulation:

- Their co-ordinates are already super-imposable, but if you have to create super-imposable co-ordinates of the ligand and protein you need to do molecular docking first.
- Hydrogens have been added to protein and crystallographic waters.
Hydrogen atoms are often not resolved in experimental data but play a critical role in forming hydrogen bonds, which are crucial for the stability and function of biomolecules.
- The protein's termini are capped with N-methyl and acetyl groups to prevent unphysical charges.
- Atoms missing from the crystal structure have been replaced.

<br />
<details>
<summary><small>▼ Click here for the shell commands we used to download the protein-ligand complex</small></summary>
    
```shell
# Clone the repository
git clone https://github.com/MCompChem/fep-benchmark.git
# Take the last ligand from the hif2a benchmark
tail -n90 fep-benchmark/hif2a/ligands.sdf > PT2385.sdf
# Take the prepared protein structure
cp fep-benchmark/hif2a/5tbm_prepared.pdb .
```  

</details>

[provides data]: https://github.com/MCompChem/fep-benchmark
[PT2385]: https://doi.org/10.1158/0008-5472.CAN-16-0473

In [ ]:
receptor_path = "5tbm_prepared.pdb"
ligand_path = "PT2385.sdf"

We can visualize each structure using the [NGLView] widget. These visualizations are interactive; rotate by dragging the left mouse button, pan with the right mouse button, and zoom with the scroll wheel. You can also mouse over an atom to see its details, and click an atom to center the view on it. When you mouse over the widget, a full screen button will appear in its top right corner.

[NGLView]: https://github.com/nglviewer/nglview

In [ ]:
view = nglview.show_structure_file(ligand_path)
view

NGLWidget()

<div class="alert alert-info" style="max-width: 700px; margin-left: auto; margin-right: auto;">
    ℹ️ Try replacing <code>ligand_path</code> with <code>receptor_path</code> to visualize the protein!
</div>


# The plan:

| Action | Software|
|--|--|
| Assemble and solvate the topology | PDBFixer, MDTraj, and OpenFF Toolkit
| Visualize the complex | OpenFF Toolkit and NGLView
| Parametrize the complex | OpenFF Toolkit and OpenFF Interchange
| Simulate the complex | OpenMM
| Visualize the simulation | NGLView and MDTraj


## Assemble and solvate the topology (PDBFixer, MDTraj, and the toolkit)

Conceptually, this step involves putting together the positions of all of the components of the system. We'll create  a [`Topology`] to keep track of the contents of our system. A `Topology` represents a collection of molecules; it doesn't have any association with any force field parameters.

[`Topology`]: https://docs.openforcefield.org/projects/toolkit/en/stable/api/generated/openff.toolkit.topology.Topology.html

First, we'll load the ligand into an OpenFF Toolkit [`Molecule`] object, which keep track of all their chemical information. A `Molecule` represents a collection of atoms with specified formal charges, connected by bonds with specified bond orders, optionally including any number of conformer coordinates. This is intended to closely align with a chemist's intuitive understanding of a molecule, rather than simply wrap the minimal information needed for a calculation.

SDF files include all a molecule's bond orders and formal charges, as well as coordinates, so they're ideal as a format for distributing small molecules. And that's exactly the format the ligand is stored in!

[`Molecule`]: https://docs.openforcefield.org/projects/toolkit/en/stable/api/generated/openff.toolkit.topology.Molecule.html

In [ ]:
# Load a molecule from a SDF file
ligand = Molecule.from_file(ligand_path, allow_undefined_stereo=True)

# Print out a SMILES code for the ligand
print(ligand.to_smiles(explicit_hydrogens=False))

# Visualize the molecule
ligand.visualize(show_all_hydrogens=False)

An SDF file with an entire protein would be huge, and finding or constructing one in the first place would be tricky, so for polymers the Toolkit supports inferring chemical information from PDB files. Connectivity can be taken from residue and atom names for supported molecules, and we can supply a list of arbitrary molecules that are in the PDB which are then identified via CONECT records.

First we'll solvate our system. [PDBFixer] makes it easy to add water and salt to our original receptor PDB, which already includes crystallographic waters. PDBFixer can also manage a number of other common processes, like restoring missing hydrogens or other atoms and even adding a membrane. We can then save it to disk and load the resulting PDB into a topology!

[PDBFixer]: https://htmlpreview.github.io/?https://github.com/openmm/pdbfixer/blob/master/Manual.html

In [ ]:
fixer = PDBFixer(receptor_path)
fixer.addSolvent(
    padding=1.0 * openmm_unit.nanometer, ionicStrength=0.15 * openmm_unit.molar
)

with open("receptor_solvated.pdb", "w") as f:
    openmm.app.PDBFile.writeFile(fixer.topology, fixer.positions, f)

top = Topology.from_pdb("receptor_solvated.pdb")

In [ ]:
def check_for_large_molecules(topology, atom_count_threshold=100):
    """Verification step. Check if there are any large molecules in the topology,
    which might indicate the presence of proteins or polymers."""
    found_large_molecule = False
    for molecule in topology.molecules:
        if len(molecule.atoms) > atom_count_threshold:
            print(f"Found a large molecule with {len(molecule.atoms)} atoms, which might be a protein or polymer.")
            found_large_molecule = True
            break

    if not found_large_molecule:
        print("No large molecules found that could indicate a protein or polymer.")
    else:
        print("Possible protein or polymer present in the topology.")

# Example usage
check_for_large_molecules(top)

In [ ]:
def insert_molecule_and_remove_clashes(
    topology: Topology,
    insert: Molecule,
    radius: Quantity = 1.5 * unit.angstrom,
    keep: Iterable[Molecule] = [],
) -> Topology:
    """
    Add a molecule to a copy of the topology, removing any clashing molecules.

    The molecule will be added to the end of the topology. A new topology is
    returned; the input topology will not be altered. All molecules that
    clash will be removed, and each removed molecule will be printed to stdout.
    Users are responsible for ensuring that no important molecules have been
    removed; the clash radius may be modified accordingly.

    Parameters
    ==========
    top
        The topology to insert a molecule into
    insert
        The molecule to insert
    radius
        Any atom within this distance of any atom in the insert is considered
        clashing.
    keep
        Keep copies of these molecules, even if they're clashing
    """
    # We'll collect the molecules for the output topology into a list
    new_top_mols = []
    # A molecule's positions in a topology are stored as its zeroth conformer
    insert_coordinates = insert.conformers[0][:, None, :]
    for molecule in topology.molecules:
        if any(keep_mol.is_isomorphic_with(molecule) for keep_mol in keep):
            new_top_mols.append(molecule)
            continue
        molecule_coordinates = molecule.conformers[0][None, :, :]
        diff_matrix = molecule_coordinates - insert_coordinates

        # np.linalg.norm doesn't work on Pint quantities 😢
        working_unit = unit.nanometer
        distance_matrix = (
            np.linalg.norm(diff_matrix.m_as(working_unit), axis=-1) * working_unit
        )

        if distance_matrix.min() > radius:
            # This molecule is not clashing, so add it to the topology
            new_top_mols.append(molecule)
        else:
            print(f"Removed {molecule.to_smiles()} molecule")

    # Insert the ligand at the end
    new_top_mols.append(ligand)

    # This pattern of assembling a topology from a list of molecules
    # ends up being much more efficient than adding each molecule
    # to a new topology one at a time
    new_top = Topology.from_molecules(new_top_mols)

    # Don't forget the box vectors!
    new_top.box_vectors = topology.box_vectors
    return new_top

top = insert_molecule_and_remove_clashes(top, ligand)

In [ ]:
# Verify that protein is present and was not removed due to clashes
check_for_large_molecules(top)

Now that we've assembled our topology, we can save it to disk. We can use JSON for this, which makes it human readable in a pinch. This stores everything we've just assembled - molecular identities, conformers, box vectors, and everything else. The topology can then be loaded later on with the [`Topology.from_json()`] method. This is great for running the same system through different force fields, distribution with a paper, or for assembling systems in stages.

[`Topology.from_json()`]: https://docs.openforcefield.org/projects/toolkit/en/stable/api/generated/openff.toolkit.topology.Topology.html#openff.toolkit.topology.Topology.from_json

<div class="alert alert-info" style="max-width: 700px; margin-left: auto; margin-right: auto;">
    ℹ️ JSON export is pretty slow, and we don't need it for the rest of the notebook, so feel free to skip it.
</div>


In [ ]:
with open("topology.json", "w") as f:
    print(top.to_json(), file=f)

In [ ]:
top = Topology.from_json(open("topology.json").read())

## Visualize the complex (NGLView)

To visualize inside the notebook, we'll use NGLView. NGLView supports a wide variety of [molecular visualization methods], as well as a VMD-like [atom selection language]. This can be used to visualize complex systems like this one.

The widget consists of a minimally documented [Python library frontend] and an extensively documented [JavaScript backend]. You'll need to refer to the documentation for both to do anything sophisticated, as the Python code delegates most of its options and functionality to the JS code.

We'll start by defining a function to create an NGLView widget from a `Topology`. This functionality will eventually find its way into the Toolkit, but for now it's here.

[molecular visualization methods]: https://nglviewer.org/ngl/api/manual/molecular-representations.html
[atom selection language]: https://nglviewer.org/ngl/api/manual/selection-language.html
[Python library frontend]: https://nglviewer.org/nglview/latest/api.html
[JavaScript backend]: https://nglviewer.org/ngl/api/manual/index.html

In [ ]:
def visualize(topology):
    """Visualize a topology with nglview"""
    with StringIO() as f:
        topology.to_file(file=f)
        pdb_str = f.getvalue()
    return nglview.show_text(pdb_str)

In [ ]:
# Create the widget. By default, proteins are shown as a cartoon and unrecognised ligands with a ball-and-stick model
view = visualize(top)

# Add a licorice/stick representation for everything except the protein
view.add_line(selection="(not protein)")
# Make the ions clearer by drawing their vdW surfaces
view.add_spacefill(selection=":.NA or :.CL")

# Render the widget
view

<div class="alert alert-info" style="max-width: 700px; margin-left: auto; margin-right: auto;">
ℹ️ Have a play with this visualization! Try clearing the default representations with <code>view.clear()</code> and configuring your own cartoon <em>(Hint: <a href=https://nglviewer.org/nglview/latest/api.html#nglview.NGLWidget>Check</a> the <a href=https://nglviewer.org/ngl/api/manual/molecular-representations.html>docs</a>)</em>. See if you can display the ligand in a way you like, without also displaying the protein's terminal caps. When you're happy with what you've made, save the image with <code>view.download_image()</code>
</div>

## Assemble the force field (OpenFF Toolkit)

Now that we've prepared our co-ordinates, we should choose the force field. For now, we don't have any single SMIRNOFF force field that can handle both proteins and small molecules; the Rosemary 3.0.0 force field will support this, but it's not expected until 2023. As an alternative, we'll combine the Sage small molecule force field with the SMIRNOFF port of Amber ff14SB. These force fields parametrize non-bonded parameters in similar ways and with the same functional form, so we don't expect any outrageous artifacts, but they also haven't been carefully tested together.

The star of the show here is the Sage 2.0.0 force field. [Sage] is the latest force field produced by the Open Force Field Initiative. Rather than using atom types like traditional biomolecular force fields, Sage assigns parameters to a molecule by [matching actual chemical moieties]. Note that to parametrize a molecule you need more than just the co-ordinates of its atoms; you also need their bonds, bond orders, and formal charges. As a result, `.sdf` files are used in this example; other file types are possible, but they must include this information. Sage is distributed as an `.offxml` file according to the [SMIRNOFF specification]. This file describes all the choices the toolkit has to make to parametrize a molecule. Sage also includes the TIP3P water model, which is appropriate for Amber ff14SB too.

When we combine multiple SMIRNOFF force fields into one, we provide them in an order from general to specific. Sage includes parameters that could be applied to a protein, but they're general across all molecules; ff14SB's parameters are specific to proteins. Since the Toolkit always applies the last parameters that match a moiety, this order makes sure the right parameters get assigned.

[Sage]: https://openforcefield.org/force-fields/force-fields/#sage
[matching actual chemical moieties]: https://www.daylight.com/dayhtml/doc/theory/theory.smarts.html
[SMIRNOFF specification]: https://openforcefield.github.io/standards/standards/smirnoff/

<div class="alert alert-warning" style="max-width: 700px; margin-left: auto; margin-right: auto;">
    🚧 <b> This code is not production-ready</b><br />
The Amber ff14SB port is intended as a proof-of-concept for SMIRNOFF protein force fields. It may not precisely match the energetics or forces of the original ff14SB, and it may not be appropriate in combination with Sage. Wait for protein support in the Rosemary force field to use this in production.
</div>


In [ ]:
sage_ff14sb = ForceField("openff-2.0.0.offxml", "ff14sb_off_impropers_0.0.3.offxml")

## Parametrize the complex (OpenFF Toolkit)

We now have a `Topology`, which stores the chemical information of the system, and a `ForceField`, which maps chemistry to force field parameters. To parametrize the system, we combine these two objects into an [`Interchange`].

An `Interchange` represents a completely parametrized molecular mechanics system. Partial charges are computed here according to the instructions in the force field, and this is where virtual sites required by the force field will be introduced. This all happens behind the scenes; all we have to do is combine an abstract chemical description with a force field. This makes it easy to change water model or force field, as the chemistry being modelled is completely independent of the model itself.

[`Interchange`]: https://docs.openforcefield.org/projects/interchange/en/stable/_autosummary/openff.interchange.components.interchange.Interchange.html

In [ ]:
interchange = sage_ff14sb.create_interchange(top)

*(Creating an interchange takes a few minutes, largely because of the complexity of the Amber protein force field port. In the future, this should be faster.)*

To use an `Interchange`, we need to convert it to the input expected by a particular molecular mechanics engine. We'll use OpenMM, because its support is the most mature and the fastest, but GROMACS, LAMMPS, and Amber all enjoy [preliminary support].

[preliminary support]: https://docs.openforcefield.org/projects/interchange/en/stable/using/output.html

In [ ]:
omm_system = interchange.to_openmm()
omm_top = interchange.to_openmm_topology()

In [ ]:
from openmm.app import *
from openmm import *
with open('system.xml', 'w') as output:
    output.write(XmlSerializer.serialize(omm_system))

_(This'll take a minute)_

While that runs, let's recap. We've constructed a `Topology` out of a number of `Molecule` objects, each of which represents a particular chemical independent of any model details. The `Topology` then represents an entire chemical system, which in theory could be modelled in any number of ways. Our `Topology` also includes atom positions and box vectors, but if we thought that was too concrete for our use case we could leave them out and add them after parametrization.

Separately, we've constructed a `ForceField` by combining a general SMIRNOFF force field with a protein-specific SMIRNOFF force field. A SMIRNOFF force field is a bunch of rules for applying force field parameters to chemicals via SMARTS patterns. The force field includes everything needed to compute an energy: parameters, charges, functional forms, non-bonded methods and cutoffs, virtual sites, and so on.

Then, we've parametrized our `Topology` with our `ForceField` to produce an `Interchange`. This applies all our rules and gives us a system ready to simulate. An `Interchange` can also concretely define positions, velocities, and box vectors, whether they come from the `Topology` or are added after parametrization. Once we have the `Interchange`, we can produce input data for any of the supported MM engines.

This clear delineation makes benchmarking the same system against different force fields or the same force field against different force fields easy. The SMIRNOFF format makes distributing force fields in an engine agnostic way possible. Everything is an open standard or written in open source Python, so we can see how it works and even change it if we need to.

## Simulate the complex (OpenMM)

All that remains is to tell OpenMM the details about how we want to integrate and record data for the simulation, and then to put everything together and run it!

### Configure and run the simulation

Here, we'll use a Langevin thermostat at 300 Kelvin and a 2 fs time step. We'll write the structure to disk every 10 steps.

In [ ]:
# Construct and configure a Langevin integrator at 300 K with an appropriate friction constant and time-step
integrator = openmm.LangevinIntegrator(
    300 * openmm_unit.kelvin, # Temperature
    1 / openmm_unit.picosecond, # Friction coefficient. This sets the speed at which the simulation's molecules adjust to the surrounding temperature.
    0.002 * openmm_unit.picoseconds, # Time step = 2fs
)

with open('system.xml') as input:
    system = XmlSerializer.deserialize(input.read())

# Combine the topology, system, integrator and initial positions into a simulation
simulation = Simulation(omm_top, system, integrator)
simulation.context.setPositions(top.get_positions().to_openmm())

# Add a reporter to record the structure every 10 steps
dcd_reporter = DCDReporter("trajectory.dcd", 10, enforcePeriodicBox=True)
simulation.reporters.append(dcd_reporter)

In [ ]:
# Save protein-ligand complex to a pdb file
from openmm.app import PDBFile

# Assume your simulation object is already created and initialized with positions
state = simulation.context.getState(getPositions=True)
positions = state.getPositions()

# Write the positions to a PDB file
with open('complex_starting_condition.pdb', 'w') as file:
    PDBFile.writeFile(simulation.topology, top.get_positions().to_openmm(), file)

### Minimize the combined system

This will reduce any forces that are too large to integrate, such as from clashes or from disagreements between the crystal structure and force field.


In [ ]:
simulation.minimizeEnergy(
    tolerance=openmm_unit.Quantity(
        value=50.0, unit=openmm_unit.kilojoule_per_mole / openmm_unit.nanometer
    )
)
minimized_state = simulation.context.getState(
    getPositions=True, getEnergy=True, getForces=True
)

print(
    "Minimised to",
    minimized_state.getPotentialEnergy(),
    "with maximum force",
    max(
        np.sqrt(v.x * v.x + v.y * v.y + v.z * v.z) for v in minimized_state.getForces()
    ),
    minimized_state.getForces().unit.get_symbol(),
)

minimized_coords = minimized_state.getPositions()

Minimised to -1702590.2784599846 kJ/mol with maximum force 2489.2505194644796 kJ/(nm mol)


In [ ]:
# Assume 'simulation' is your Simulation object
# and 'minimized_coords' contains the positions from the minimized state

# Get the topology from your simulation object
topology = simulation.topology


# Use the PDBFile class to write the topology and minimized coordinates to a PDB file
with open('minimized_structure.pdb', 'w') as outfile:
    PDBFile.writeFile(topology, minimized_coords, outfile)

print("Minimized structure saved to minimized_structure.pdb")


Minimized structure saved to minimized_structure.pdb


In [ ]:
# saveState saves the state of simulation including position for later use.
simulation.saveState('minimized_state.xml')

## Equilibrate system to desired temperature

In [ ]:
from openmm.unit import nanometers, kelvin, picoseconds, picosecond, atmospheres

# Equilibration under NVT conditions. The number of particles and volume is fixed and temperature is stabilized with a thermostat.
temperature = 300*kelvin # Target temperature
friction = 1/picosecond  # The friction coefficient
time_step = 0.001*picoseconds  # Time step for integration


# Combine the topology, system, integrator and initial positions into a simulation
nvt_integrator = openmm.LangevinIntegrator(temperature, friction, time_step)
nvt_simulation = Simulation(omm_top, system, nvt_integrator)
nvt_simulation.loadState('minimized_state.xml')
nvt_simulation.context.setVelocitiesToTemperature(temperature) # adjust velocities to control temperature

# Set up reporters to monitor equilibration
nvt_simulation.reporters.append(StateDataReporter("nvt_equilibration.log",
                                                200, # number of steps between each save
                                                step=True, # writes step number to each line
                                                potentialEnergy=True,  # writes potential energy of the system (KJ/mole)
                                                temperature=True
                                                ))

# Equilibrate for a certain number of steps
nvt_steps = 4000  # Adjust according to your system's requirements
print("Equilibrating...")
nvt_simulation.step(nvt_steps) # Run MD simulation
print("Equilibration complete.")
simulation.saveState('nvt_equilibration.xml')

Equilibrating...
Equilibration complete.


In [ ]:
# Function to parse the log file
def parse_log_file(log_file_path):
    print("Parsing log file")
    steps = []
    energies = []
    temperatures = []  # List to store temperature values
    with open(log_file_path, 'r') as file:
        for line in file:
            # Skip header or lines without numeric data
            try:
                step, energy,temp = line.strip().split(',')
                steps.append(int(step))
                energies.append(float(energy))
                temperatures.append(float(temp))  # Parse temperature
            except ValueError:
                continue
    return steps, energies, temperatures

# Path to your log file
log_file_path = 'nvt_equilibration_short2.log'

# Parse the log file
steps, energies, temperatures = parse_log_file(log_file_path)

# Create figure and first axis
plt.figure(figsize=(10, 6))
ax1 = plt.gca()  # Get current axis
ax2 = ax1.twinx()  # Create another axis that shares the same x-axis

# Plot energy on the first y-axis
ax1.plot(steps, energies, marker='o', linestyle='-', color='blue', label='Energy')
ax1.set_xlabel('Time Step')
ax1.set_ylabel('Energy', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# Plot temperature on the second y-axis
ax2.plot(steps, temperatures, marker='x', linestyle='-', color='red', label='Temperature')
ax2.set_ylabel('Temperature', color='red')
ax2.tick_params(axis='y', labelcolor='red')

# Title and grid
plt.title('Energy and Temperature vs. Time Step')
ax1.grid(True)

# Optional: add a legend. Comment these lines if you find the legend unnecessary.
ax1.legend(loc='upper left')
# ax2.legend(loc='upper right')

plt.show()


## Equilibrate system to desired pressure

In [ ]:
from openmm import XmlSerializer, MonteCarloAnisotropicBarostat
from openmm.app import Simulation, StateDataReporter, PDBReporter
from openmm import LangevinIntegrator
from openmm.unit import kelvin, picoseconds, atmospheres
import numpy as np

class CheckpointReporter:
    def __init__(self, file, reportInterval):
        self._reportInterval = reportInterval
        self._out = open(file, 'wb')

    def describeNextReport(self, simulation):
        steps = self._reportInterval - simulation.currentStep % self._reportInterval
        return (steps, False, False, False, False)

    def report(self, simulation, state):
        simulation.saveCheckpoint(self._out.name)


# Load your system
with open('system.xml') as input:
    npt_system = XmlSerializer.deserialize(input.read())

# Temperature and pressure settings
temperature = 300*kelvin
pressure_x = 1*atmospheres
pressure_y = 1*atmospheres
pressure_z = 1*atmospheres
barostat_frequency = 25

# Setting up anisotropic pressures
# The pressures are specified as a list or tuple in the order (Px, Py, Pz)
# The boolean flags indicate whether the barostat should adjust the box size in that direction
pressure = (pressure_x, pressure_y, pressure_z)
scaleXYZ = (True, True, True)  # Allow box size to vary in X, Y, and Z directions

# Add MonteCarloAnisotropicBarostat to the system
barostat = MonteCarloAnisotropicBarostat(pressure, temperature, scaleXYZ[0], scaleXYZ[1], scaleXYZ[2], barostat_frequency)
npt_system.addForce(barostat)

# # Integrator settings
friction = 1/picoseconds  # Friction coefficient for Langevin Integrator
time_step = 0.002*picoseconds  # Integration timestep

# Create a Langevin Integrator
npt_integrator = LangevinIntegrator(temperature, friction, time_step)

# Set up the Simulation
npt_simulation = Simulation(omm_top, npt_system, npt_integrator)
# npt_simulation.loadState('nvt_equilibration_short2.xml')
# npt_simulation.context.setVelocitiesToTemperature(temperature)

# Load the simulation state from a checkpoint file
checkpoint_file = 'checkpoint_30K.chk'  # Use the actual path to your checkpoint file
npt_simulation.loadCheckpoint(checkpoint_file)

# # Optionally, add reporters to monitor the simulation
# npt_simulation.reporters.append(PDBReporter('equilibration.pdb', 100))  # Save a PDB every 1000 steps
npt_simulation.reporters.append(StateDataReporter('equilibration_30K.log', 100, step=True, temperature=True, potentialEnergy=True, totalEnergy=True, density=True, volume=True))  # Log every 1000 steps
npt_simulation.reporters.append(DCDReporter('equilibration_30K.dcd', 100))
npt_simulation.reporters.append(CheckpointReporter('equilibration_checkpoint.chk', 100))  # Save a checkpoint every 1000 steps


# Run the equilibration
npt_simulation.step(30000)


In [ ]:
# Save results. Checkpoint saves the state incl. position and velocities so you can continue your simulation where you left off.
npt_simulation.saveState('npt_equillibration_30K.xml')
npt_simulation.saveCheckpoint('checkpoint_30K.chk')

In [ ]:
# Function to parse the log file
def parse_log_file(log_file_path):
    print("Parsing log file")
    steps = []
    energies = []
    temperatures = []  # List to store temperature values
    volumes = []
    with open(log_file_path, 'r') as file:
        for line in file:
            # Skip header or lines without numeric data
            try:
                step, pot_energy, energy, temp, volume, density = line.strip().split(',')
                steps.append(int(step))
                energies.append(float(energy))
                temperatures.append(float(temp))
                volumes.append(float(volume))
            except ValueError:
                continue
    return steps, energies, temperatures, volumes

# Path to your log file
log_file_path = 'equilibration.log'

# Parse the log file
steps, energies,temperatures, volumes = parse_log_file(log_file_path)
print(steps)

# # Create figure and first axis
plt.figure(figsize=(10, 6))
ax1 = plt.gca()  # Get current axis
ax2 = ax1.twinx()  # Create another axis that shares the same x-axis

# # Plot energy on the first y-axis
ax1.plot(steps, energies, marker='o', linestyle='-', color='blue', label='Energy')
ax1.set_xlabel('Time Step')
ax1.set_ylabel('Energy', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# # Plot temperature on the second y-axis
ax2.plot(steps, volumes, marker='x', linestyle='-', color='red', label='Volume')
ax2.set_ylabel('Volume', color='red')
ax2.tick_params(axis='y', labelcolor='red')

# # Title and grid
plt.title('Energy and Temperature vs. Time Step')
ax1.grid(True)

# # Optional: add a legend. Comment these lines if you find the legend unnecessary.
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')

plt.show()

### Run a short simulation

If this were anything more than a demonstration of the Toolkit, this example would need to include additional steps like equilibration.

<div class="alert alert-warning" style="max-width: 700px; margin-left: auto; margin-right: auto;">
⚠️ Make sure you use your own, valid simulation protocol! This is just an example.
</div>

In [ ]:
simulation.context.setVelocitiesToTemperature(300 * openmm_unit.kelvin)
simulation.runForClockTime(1.0 * openmm_unit.minute)

In [ ]:
import mdtraj as md

traj = md.load_dcd('trajectory_5t.dcd', top='minimized_structure_v3.pdb')
traj.image_molecules(inplace=True)  # This re-wraps or images the molecules

traj.save_dcd('trajectory_image.dcd')  # Save the processed trajectory

traj.save_dcd('trajectory_image.pdb')

_(This'll take a minute)_

While that runs, let's talk a bit about OpenFF

### Open Source Force Fields

A primary goal of the Open Force Field Initiative is to make development and use of force fields as open as possible - it's in our name! We believe that open source development practices have a lot to offer the scientific community, whether that science is academic, commercial, or hobbyist.

#### The SMIRNOFF specification

The SMIRNOFF specification describes a simple format for describing molecular force fields. We provide and maintain this spec in the hopes that it will allow scientists everywhere to contribute to force field development in a unified way, without taking them away from their favourite simulation package.

SMIRNOFF is not just a spec; we're also committed to a reference implementation — that being the OpenFF Toolkit. The Toolkit endeavors to support all the functional forms in both the SMIRNOFF spec and the [`openff-forcefields`](https://github.com/openforcefield/openff-forcefields/) package.

#### Reproducibility

OpenFF force fields are completely specified by the name of the distributed `.offxml` file. We use codenames, version numbers, and tags to accomplish this. This means that as long as a user, designer, or reviewer sees the name of the force field being used, they know exactly what is going in to that simulation. We include parameters that are often neglected in force field specifications, such as the non-bonded cut-off distance, ewald methods, constraints, modifications to the Lennard-Jones function, and partial charge generation methods are all defined by the name of the force field.

As much as possible, we want energy and force to be a deterministic output of combining a molecule and a force field. If an author provides the name of the force field in their methods section, it should be reproducible. The other side of this coin is that we never want to hide the force field from the user. In all our workflows, the name of the force field must be explicitly provided in the code. This improves reproducibility of the code and helps the user take responsibility for their results.

#### "Plugin" support for new force fields

The OpenFF Toolkit supports distributing force field files (.offxml) through Conda data packages. Anyone can publish a package on Conda Forge that extends the list of directories the toolkit searches for force fields, allowing anyone to produce force fields without requiring their own tooling, in a format that is designed to be converted to a multitude of simulation packages. See the [FAQ](https://open-forcefield-toolkit.readthedocs.io/en/stable/faq.html#how-can-i-distribute-my-own-force-fields-in-smirnoff-format) for more details.

---

Right! Simulation should be done by now, let's take a look.

## Visualize the simulation (nglview)

NGLView can display trajectories, as well as single structures. Mouse over the widget to see the animation controls.

In [ ]:
trajectory: mdtraj.Trajectory = mdtraj.load(
    "trajectory.dcd", top=mdtraj.Topology.from_openmm(omm_top)
)

view = nglview.show_mdtraj(trajectory.image_molecules())
view.add_representation("line", selection="protein")
view

<div class="alert alert-info" style="max-width: 700px; margin-left: auto; margin-right: auto;">
ℹ️ MDTraj is a great library for analysis as well as visualisation. Check out the <a href=https://mdtraj.org/1.9.4/api/generated/mdtraj.Trajectory.html>docs</a> for the <code>Trajectory</code> object you just created, as well as their <a href=https://mdtraj.org/1.9.4/analysis.html>analysis functions</a>, and see if you can compute something interesting. Its real superpower is that it provides the coordinates of the trajectory as a <a href=https://numpy.org/doc/stable/reference/generated/numpy.array.html>NumPy array</a>, so if you're really keen try computing something directly from <code>mdt_traj.xyz</code>
</div>

## Conclusions

* The OpenFF workflow cleanly separates the chemical system from its model.
* We parametrize ligands and proteins with the same software tools.
* Open source tools installed via Conda did everything, from basic system prep to simulation and visualization
* Using OpenMM, we never had to leave Python to set up the simulation
* With Interchange, using OpenMM, GROMACS, Amber or LAMMPS is simple!


## What's next?

We have more examples for you to explore the toolkit!

 - [Force Field Modification](https://github.com/openforcefield/openff-toolkit/blob/main/examples/forcefield_modification/): Use the Toolkit API to manipulate SMIRNOFF parameters in a `ForceField` object
 - [Conformer energies](https://github.com/openforcefield/openff-toolkit/blob/main/examples/conformer_energies/): Compute vacuum energies of different conformers of a small molecule with the Sage force field
 - [QCArchive interface](https://github.com/openforcefield/openff-toolkit/blob/main/examples/QCArchive_interface/): Create OpenFF `Molecule` objects from the [QCArchive](https://qcarchive.molssi.org/)